# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write complex queries
* Write subqueries to decompose complex queries

## CRM Database Schema

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database **data.sqlite**.

In [1]:
import pandas as pd
import sqlite3
conn = sqlite3.connect('data.sqlite')
c = conn.cursor()

## Write an Equivalent Query using a Subquery

```SQL
select customerNumber,
       contactLastName,
       contactFirstName
       from customers
       join orders 
       using(customerNumber)
       where orderDate = '2003-01-31';
```

In [2]:

query = """
SELECT  customerNumber,
        contactLastName,
        contactFirstName
        FROM customers
        WHERE customerNumber IN (SELECT customerNumber
                                 FROM orders
                                 WHERE orderDate = '2003-01-31')
        """

# query_original = """
# select customerNumber,
#        contactLastName,
#        contactFirstName
#        from customers
#        join orders 
#        using(customerNumber)
#        where orderDate = '2003-01-31';
#        """


df = pd.DataFrame(c.execute(query).fetchall())
df.columns = [x[0] for x in c.description]
df

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [3]:
query = """
        SELECT  p.productName, IFNULL(SUM(od.quantityOrdered),0) as total_quantity_ordered
        FROM products p
        LEFT JOIN orderdetails od
        ON od.productCode = p.productCode
        GROUP BY p.productName
        ORDER BY total_quantity_ordered DESC

"""
df = pd.DataFrame(c.execute(query).fetchall())
df.columns = [x[0] for x in c.description]
df

,productName,total_quantity_ordered
0,1992 Ferrari 360 Spider red,1808
1,1937 Lincoln Berline,1111
2,American Airlines: MD-11S,1085
3,1941 Chevrolet Special Deluxe Cabriolet,1076
4,1930 Buick Marquette Phaeton,1074
5,1940s Ford truck,1061
6,1969 Harley Davidson Ultimate Chopper,1057
7,1957 Chevy Pickup,1056
8,1964 Mercedes Tour Bus,1053
9,1956 Porsche 356A Coupe,1052


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

### A quick note on the SQL  `SELECT DISTINCT` statement:

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [4]:
#Your code here:
# Hint: because one of the tables we'll be joining has duplicate customer numbers, you should use DISTINCT
query = """
        SELECT  COUNT(DISTINCT o.customerNumber) AS num_customers_ordered, p.productName
        FROM orders o
        JOIN orderdetails od
        USING(orderNumber)
        JOIN products p
        USING (productCode)
        GROUP BY productCode
        ORDER BY num_customers_ordered DESC
        """

df = pd.DataFrame(c.execute(query).fetchall())
df.columns = [x[0] for x in c.description]
print(len(df))
df.head()

#                 c.contactFirstName, 
#                 c.contactLastName,
#                 od.productCode

109


,num_customers_ordered,productName
0,40,1992 Ferrari 360 Spider red
1,27,1952 Alpine Renault 1300
2,27,1972 Alfa Romeo GTA
3,27,1934 Ford V8 Coupe
4,27,Boeing X-32A JSF


## Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products Which Have Been Ordered by Less Then 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [5]:
query = """
              SELECT DISTINCT e.firstName, e.Lastname, e.employeeNumber, o.city, o.officeCode
              FROM employees e
              JOIN offices o
              USING(officeCode)
              JOIN customers c
              ON e.employeeNumber = c.salesRepEmployeeNumber
              JOIN orders o
              USING(customerNumber)
              JOIN orderdetails od
              USING(orderNumber)
              JOIN products p
              USING(productCode)
              WHERE productCode IN(
                                    SELECT  p.productCode 
                                    from products p
                                    JOIN orderdetails od
                                    USING (productCode)
                                    JOIN orders o
                                    USING(orderNumber)
                                    JOIN customers c
                                    USING(customerNumber)
                                    GROUP BY p.productCode
                                    HAVING COUNT(DISTINCT c.customerNumber) < 20
                              
                                    
                                    )


        """
df = pd.DataFrame(c.execute(query).fetchall())
df.columns = [x[0] for x in c.description]
print(len(df))
df

#             WHERE e.employeeNumber IN
#             (SELECT c.salesRepEmployeeNumber 
#              FROM customers c
#              JOIN orders o
#              USING(customerNumber)
#              JOIN orderdetails od
#              USING(orderNumber)
#              GROUP BY c.salesRepEmployeeNumber
#              HAVING COUNT(productCode) < 20);


# query = """
#             SELECT  e.employeeNumber, 
#                     e.firstName, 
#                     e.lastName, 
#                     o.officeCode, 
#                     o.city
#             FROM employees e
#             JOIN offices o
#             USING(officeCode)

#         """

15


,firstName,lastName,employeeNumber,city,officeCode
0,Gerard,Hernandez,1370,Paris,4
1,Larry,Bott,1501,London,7
2,Loui,Bondur,1337,Paris,4
3,Leslie,Thompson,1166,San Francisco,1
4,Foon Yue,Tseng,1286,NYC,3
5,Peter,Marsh,1612,Sydney,6
6,Andy,Fixter,1611,Sydney,6
7,Pamela,Castillo,1401,Paris,4
8,Mami,Nishi,1621,Tokyo,5
9,George,Vanauf,1323,NYC,3


## Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Who's Customers Have an Average Credit Limit of Over 15K

In [20]:
#Your code here
query = """
        SELECT COUNT(c.customerNumber) as num_customers
        FROM employees e
        JOIN customers c
        ON e.employeeNumber = c.salesRepEmployeeNumber
        GROUP BY employeeNumber
        HAVING SUM(c.creditLimit) > 15000
ORDER BY e.employeeNumber

        """

df = pd.DataFrame(c.execute(query).fetchall())
df.columns = [x[0] for x in c.description]
print(len(df))
df
# df['num_customers'].sum()

# query = """
#         SELECT * FROM customers c
        
#         ORDER BY creditLimit
#         """



# query = """
#         SELECT e.employeeNumber, e.firstName, e.lastName, COUNT(c.customerNumber) 
#         FROM employees e
#         JOIN customers c
#         ON e.employeeNumber = c.salesRepEmployeeNumber
#         GROUP BY e.employeeNumber, e.firstName, e.lastName
#         HAVING SUM(c.creditLimit) > 15000
# ORDER BY e.employeeNumber

#         """

15


,num_customers
0,6
1,6
2,6
3,6
4,7
5,8
6,6
7,7
8,10
9,8


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!